In [143]:
import pandas as pd
import numpy as np 
import nltk
import random
from patsy import dmatrices
#nltk.download()
from string import punctuation
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\teeru\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [144]:
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set (stopwords.words( 'english' ))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\teeru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [145]:
data = pd.read_csv("D:/UTA/Fall term/Text analytics/HW1/Train_rev1.csv")
df = data.sample(2500, random_state = 99)

In [146]:
tk =[]
tk_fil =[]
for sen in df['FullDescription']:
    #make sentence lowercase
    sen = sen.lower()
    #remove punctuation
    for p in punctuation:
        sen=sen.replace(p,'') 
    #tokenize to words
    txt = word_tokenize(sen)
    #tag pos
    tk += nltk.pos_tag(txt)
    #remove stopwords
    txt_fil = [word for word in txt if word not in stop_words]
    tk_fil += nltk.pos_tag(txt_fil)

In [147]:
from collections import Counter
tk_lst = Counter(tag for (word, tag) in tk)
print (tk_lst.most_common(5))

[('NN', 156816), ('JJ', 69903), ('IN', 65426), ('NNS', 50777), ('DT', 50574)]


In [148]:
tk_lst_fil = Counter(tag for (word, tag) in tk_fil)
print (tk_lst_fil.most_common(5))

[('NN', 151968), ('JJ', 71969), ('NNS', 50279), ('VBG', 25536), ('VBP', 16812)]


A2 . Does this data support Zipf’s law? Plot the most common 100 words in the data against the theoretical prediction of the law. For this question, do not remove stopwords. Also do not perform stemming or lemmatization

In [149]:
txt_zf = []
for sen in df['FullDescription']:
    #make sentence lowercase
    sen = sen.lower()
    #remove punctuation
    for p in punctuation:
        sen=sen.replace(p,'') 
    #tokenize to words
    txt_zf += word_tokenize(sen)

In [150]:
tk_zf = Counter(txt_zf)

In [151]:
topwords = tk_zf.most_common(100)

In [152]:
frequency = []
for (word, freq) in topwords:
    frequency.append(freq)

frequency = pd.Series(frequency)

# get the observational rank# get th 
word_ranking = frequency.rank(method = 'min',ascending =False)

In [153]:
df_freq_rank = pd.DataFrame(dict(Frequency = frequency,Rank = word_ranking))

In [154]:
df_freq_rank['logrank']= np.log(df_freq_rank.Rank)

In [155]:
n = 2500*df_freq_rank['Frequency'].max()

In [156]:
df_freq_rank['logfreq']= np.log((df_freq_rank.Frequency)/n)

In [158]:
import statsmodels.api as sm
y, X = dmatrices('logrank ~ logfreq', data=df_freq_rank, return_type='dataframe')
model = sm.OLS(y, X)       # Set up the model
result = model.fit()       # Fit model (find the intercept and slopes)
print (result.summary())

                            OLS Regression Results                            
Dep. Variable:                logrank   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                     8399.
Date:                Tue, 11 Sep 2018   Prob (F-statistic):           8.78e-97
Time:                        21:49:58   Log-Likelihood:                 89.216
No. Observations:                 100   AIC:                            -174.4
Df Residuals:                      98   BIC:                            -169.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -7.8071      0.125    -62.322      0.0

A3. If we remove stopwords and lemmatize the data, what are the 10 most common words? What are their frequencies?

In [159]:
txt_lem =[]
for sen in df['FullDescription']:
    #make sentence lowercase
    sen = sen.lower()
    #remove punctuation
    for p in punctuation:
        sen=sen.replace(p,'') 
    txt_lem += word_tokenize(sen)
#remove stopwords
tk_lem = [word for word in txt_lem if word not in stop_words]

For verb

In [160]:
from nltk.stem import WordNetLemmatizer
tk_lem_proc = [WordNetLemmatizer().lemmatize(t, pos = 'v') for t in tk_lem]

In [161]:
tk_lem_lst = Counter(tk_lem_proc)
print (tk_lem_lst.most_common(10))

[('work', 5283), ('experience', 5110), ('team', 3058), ('role', 2952), ('business', 2747), ('service', 2444), ('skills', 2353), ('sales', 2181), ('within', 2181), ('client', 2006)]


For all POS

In [162]:
from nltk.corpus import wordnet

def get_pos(word):
    w_synsets = wordnet.synsets(word)
    pos_counts = Counter()
    pos_counts["n"] = len(  [ item for item in w_synsets if item.pos()=="n"]  )
    pos_counts["v"] = len(  [ item for item in w_synsets if item.pos()=="v"]  )
    pos_counts["a"] = len(  [ item for item in w_synsets if item.pos()=="a"]  )
    pos_counts["r"] = len(  [ item for item in w_synsets if item.pos()=="r"]  )
    
    most_common_pos_list = pos_counts.most_common(3)
    return most_common_pos_list[0][0]
    
tk_lem_proc = [WordNetLemmatizer().lemmatize(t, get_pos(t)) for t in tk_lem]

In [163]:
tk_lem_lst = Counter(tk_lem_proc)
print (tk_lem_lst.most_common(10))

[('work', 5283), ('experience', 5110), ('role', 3160), ('team', 3058), ('client', 2994), ('business', 2864), ('service', 2444), ('skill', 2420), ('sale', 2209), ('within', 2181)]


# Part B (predict salary from job description; the idea here is to test the predictive power of text and compare it with that of numeric data)
In this section, you will create classification models to predict high (75th percentile and above) or low (below 75th percentile) salary from the text contained in the job descriptions.

B1. Ignore the job descriptions, and train a model to predict high/low salary from all the numeric columns, e.g., part time/full time, contract vs. others, type of job (a lot of dummy variables), location (instead of using a huge number of dummy variables, you can use a list of cities in England with highest cost of living, and create a 0/1 variable which is 1 if the job is in one of those cities, else 0). Use the Naïve Bayes classifier. What is the accuracy of your model?    

In [164]:
df1 = df.iloc[:,3:-1]

In [165]:
df1['HighSalary'] = 0.0
mask = df1['SalaryNormalized'] > df1['SalaryNormalized'].quantile(.75)
df1['HighSalary'][mask] = 1.0

C:\Users\teeru\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [166]:
highlivingcost = ['oxford','uk','the city','greater london','central london','london','south east london','winchester', 'cambridge', 'chichester', 'brighon and hove', 'bath', 'southhampton','salisbury','canterbury','st albans', 'bristol','lichfield','truro','norwich','chelmsford','exeter','york','leicester','gloucester']

In [167]:
def lowerize(x):
  return x.lower()

df1['HighLocation'] = 0.0
df1['HighLocation'][df1['LocationNormalized'].apply(lowerize).isin(highlivingcost)] = 1.0
df1 = df1.drop(columns=['LocationRaw','SalaryRaw','LocationNormalized','Company', 'SalaryNormalized'])

C:\Users\teeru\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [168]:
categorical_columns = ['ContractType', 'ContractTime', 'Category']
df_dummies = pd.get_dummies(df1[categorical_columns],
                            prefix=categorical_columns,
                            columns=categorical_columns)
dummy_column_names = df_dummies.columns.values

In [169]:
df2 = pd.concat([df1, df_dummies], axis=1)

In [171]:
formula = 'HighSalary ~ 0 + {}'.format(' + '.join(['Q("{}")'.format(x) for x in dummy_column_names]))

In [172]:
from patsy import dmatrices
Y, X = dmatrices(formula, df2, return_type='dataframe')

In [173]:
y = Y['HighSalary'].values

In [174]:
from sklearn import naive_bayes
model = naive_bayes.MultinomialNB()

In [175]:
random.seed(99)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [176]:
#Baseline accuracy
print ((len(y_test) - y_test.sum()) /len(y_test))

0.7333333333333333


In [177]:
model.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [178]:
from sklearn import metrics
prediction_train = model.predict(X_test)
print (metrics.accuracy_score(y_test, prediction_train))

0.7333333333333333


In [179]:
model = naive_bayes.BernoulliNB()
model.fit(X_train, y_train)
prediction_train = model.predict(X_test)
print (metrics.accuracy_score(y_test, prediction_train))

0.72


# Text only

In [180]:
df3 = df

In [181]:
df3['HighSalary'] = 0.0
mask = df3['SalaryNormalized'] > df3['SalaryNormalized'].quantile(.75)
df3['HighSalary'][mask] = 1.0

C:\Users\teeru\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [182]:
df4 = df3[['FullDescription','HighSalary']]

# Bag of words

In [183]:
#create corpus
bow = []
for sen in df4['FullDescription']:
    sen = sen.lower()
    for p in punctuation:
        sen = sen.replace(p,"")
    bow.append(sen)
    

In [184]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
count = vectorizer.fit_transform(bow).todense() 

In [185]:
ft = vectorizer.get_feature_names()
df5 = pd.SparseDataFrame(count, columns=ft)

In [186]:
df4 = df4.reset_index()
df4 = df4[['HighSalary','FullDescription']]

In [187]:
df6 = df5.join(df4)

In [188]:
df6 = df6.drop(columns = 'FullDescription')

In [189]:
formula = 'HighSalary ~ 0 + {}'.format(' + '.join(['Q("{}")'.format(x) for x in ft]))

In [190]:
y_text = df6['HighSalary'].values

In [191]:
model = naive_bayes.MultinomialNB()

In [192]:
X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(df5, y_text, test_size=0.3)

In [193]:
model.fit(X_train_text, y_train_text)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [195]:
prediction_train_text = model.predict(X_test_text)
print (metrics.accuracy_score(y_test_text, prediction_train_text))

0.7933333333333333


# Hybrid

In [196]:
df7 = df2.drop(columns = ['ContractType','ContractTime','Category', 'HighSalary'])

In [197]:
df7 = df7.reset_index().drop(columns = 'index')

In [199]:
df8 = df6.join(df7)

In [200]:
var = df8.columns.values

In [201]:
formula = 'HighSalary ~ 0 + {}'.format(' + '.join(['Q("{}")'.format(x) for x in var]))

In [202]:
y_hybrid = df8['HighSalary'].values

In [203]:
model = naive_bayes.MultinomialNB()

In [204]:
df9 = df8.drop(columns = 'HighSalary')
X_train_hybrid, X_test_hybrid, y_train_hybrid, y_test_hybrid = train_test_split(df9, y_hybrid, test_size=0.3)

In [205]:
model.fit(X_train_hybrid, y_train_hybrid)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [206]:
prediction_train_hybrid = model.predict(X_test_hybrid)
print (metrics.accuracy_score(y_test_hybrid, prediction_train_hybrid))

0.8026666666666666
